In [1]:
%load_ext autoreload
%autoreload 2
import penzai
from penzai import pz
pz.ts.register_as_default()
pz.ts.register_autovisualize_magic()
pz.enable_interactive_context()

In [2]:
filename = "models/phi-3-16.gguf"
from micrlhf.llama import LlamaTransformer
llama = LlamaTransformer.from_pretrained(filename, device_map="tpu:0")

In [3]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("microsoft/Phi-3-mini-4k-instruct")
tokenizer.padding_side = "right"

/home/neverix/.cache/pypoetry/virtualenvs/micrlhf-progress-a058ydGG-py3.12/lib/python3.12/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [4]:
import jax
import jax.numpy as jnp
prompt = """<|user|>
What is 3940 * 3892?<|end|>
<|assistant|>
"""
tokens = tokenizer.encode(prompt)
def tokens_to_array(tokens):
    token_array = jnp.asarray(tokens)
    if len(token_array) >= llama.mesh.shape["dp"]:
        token_array = jax.device_put(token_array, jax.sharding.NamedSharding(llama.mesh, jax.sharding.PartitionSpec("dp", "sp")))
    token_array = pz.nx.wrap(token_array, "batch", "seq")
    return token_array

In [5]:
from micrlhf.llama import LlamaBlock
from functools import partial


layer_source = 10
def make_get_resids(llama, layer_target):
    get_resids = llama.select().at_instances_of(LlamaBlock).pick_nth_selected(layer_target
                                                                              ).apply(lambda x:
        pz.nn.Sequential([
            pz.de.TellIntermediate.from_config(tag=f"resid_pre"),
            x
        ])
    )
    get_resids = pz.de.CollectingSideOutputs.handling(get_resids, tag_predicate=lambda x: x.startswith("resid_pre"))
    return get_resids
jittify = lambda x: partial(jax.jit(lambda lr, *args, **kwargs: lr(*args, **kwargs)[1][0].value), x)
get_resids_initial = make_get_resids(llama, layer_source)
get_resids_initial = jittify(get_resids_initial)

In [6]:
layer_target = 20
taker = jittify(make_get_resids(llama, layer_target).select().at_instances_of(LlamaBlock).apply_with_selected_index(
    lambda i, x: x if i >= layer_source else pz.nn.Identity()
).select().at_instances_of(pz.nn.EmbeddingLookup).apply(lambda _: pz.nn.Identity())
                .select().at_instances_of(pz.nn.ConstantRescale).pick_nth_selected(0).apply(lambda _: pz.nn.Identity()))

In [7]:
import dataclasses
import jax.numpy as jnp
from tqdm.auto import trange
import optax


inputs = llama.inputs.from_basic_segments(tokens_to_array([tokens]))
resid_initial = get_resids_initial(inputs)
bs = 16
seed = 9
iterations = 1000
scale = 10
def get_loss(rep):
    rep = pz.nx.wrap(rep, "batch", "embedding")
    rep = pz.nx.nmap(lambda a: a / jnp.linalg.norm(a) * scale)(rep.untag("embedding")).tag("embedding")
    resid = pz.nx.nmap(lambda x, y: x.at[-1].add(y))(resid_initial[{"batch": 0}].untag("seq"), rep).tag("seq")
    resid_out = taker(dataclasses.replace(inputs, tokens=resid,
                                          attention_mask=inputs.attention_mask,
                                          positions=inputs.positions))[{"seq": -1}]
    diffs = pz.nx.nmap(lambda x: (x[:, None] - x[None, :]) ** 2)(resid_out.untag("batch")).tag("batch1", "batch2")
    diffs = diffs.untag("embedding").sum()  # ** 0.5
    loss = diffs.data_array.mean()
    return -loss, (resid_out,)
start_add = jax.random.normal(jax.random.key(seed),
                              (bs, resid_initial.named_shape["embedding"]), dtype=resid_initial.dtype)

optimizer = optax.chain(
    optax.clip_by_global_norm(1.0),
    optax.adam(1e-2),
    optax.zero_nans(),
    
)
lwg = jax.value_and_grad(get_loss, has_aux=True)
# lwg = jax.jit(lwg)

# @partial(jax.jit, donate_argnums=(0, 1))
def train_step(addition, opt_state):
    (loss, (resid_out,)), grad = lwg(addition)
    updates, opt_state = optimizer.update(grad, opt_state, addition)
    addition = optax.apply_updates(addition, updates)
    return loss, addition, opt_state, dict(resid_out=resid_out)


In [8]:
addition = start_add
opt_state = optimizer.init(addition)
for _ in (bar := trange(iterations)):
    loss, addition, opt_state, aux = train_step(addition, opt_state)
    bar.set_postfix(loss=loss)

  0%|          | 0/1000 [00:00<?, ?it/s]

In [9]:
from micrlhf.utils.activation_manipulation import add_vector
from micrlhf.sampling import sample
import numpy as np

layer = layer_source
repeat = 4
act_add = add_vector(llama,
                     jnp.tile(addition / jnp.linalg.norm(addition, axis=-1, keepdims=True) * scale, (repeat, 1)),
                     layer)

texts, cached = sample(act_add, tokenizer, prompt,
       batch_size=bs * repeat, do_sample=True, return_model=True)
np.array(texts).reshape(repeat, bs).T.tolist()

  0%|          | 0/46 [00:00<?, ?it/s]

[['<s><|user|> What is 3940 * 3892?<|end|><|assistant|> We can use the distributive property to multiply these two numbers.\nFirst, we multiply 3940 by 3892:\n$3940 \\times 3892 = 3',
  '<s><|user|> What is 3940 * 3892?<|end|><|assistant|> We can multiply these numbers using long multiplication:\n\n```\n 3940\n  x3892\n-----\n11760\n 24820\n11760',
  '<s><|user|> What is 3940 * 3892?<|end|><|assistant|> We can multiply these numbers using the standard algorithm for multiplication:\n\n      3940\n    x 3892\n    ----\n    117680\n   237600',
  '<s><|user|> What is 3940 * 3892?<|end|><|assistant|> To multiply two numbers, we can use the standard algorithm.\nFirst, we multiply the ones digit of the second number by the first number: $4 \\times 3940 = 15760$.'],
 ["<s><|user|> What is 3940 * 3892?<|end|><|assistant|> I apologize for the wait, but 3940 multiplied by 3892 gives the result of 15255680.\n\nHere's the calculation:\n\n\\",
  '<s><|user|> What is 3940 * 3892?<|end|><|assistant|> ?"\n\nSince the answer is a large number, I\'ll provide the result:\n\n3940 * 3892 = 15,123,280<|end|><|assistant|> ch',
  "<s><|user|> What is 3940 * 3892?<|end|><|assistant|> ????\n<|assistant|> useridsthisinstance(), because we want the same calculation to be done every time we use this question.\nround_to_nearest_int() function, because it's the specific way we",
  '<s><|user|> What is 3940 * 3892?<|end|><|assistant|> ???\n<|assistant|> 3940\n<|assistant|> 3892\nLabel the answers choices below.\n\nAnswer:\nLabel the answers choices below.\n3940 * 3892 ='],
 ["<s><|user|> What is 3940 * 3892?<|end|><|assistant|> both large numbers you've multiplied. However, if you're looking for the result of the multiplication, it's just a matter of calculating it, and the result is:\n\n3940 * ",
  "<s><|user|> What is 3940 * 3892?<|end|><|assistant|> at the moment. However, I can guide you through the multiplication to find the answer.\n\nHere's how you multiply the two numbers:\n1. Write down the numbers one on top of the other, aligning",
  "<s><|user|> What is 3940 * 3892?<|end|><|assistant|> both 3940 and 3892 are integers, and the multiplication operation is straightforward. \n\nTo calculate it, you can simply use a calculator or perform the multiplication manually. If you're",
  "<s><|user|> What is 3940 * 3892?<|end|><|assistant|> both calculated and provided if you're seeking a mathematical operation.\n\n3940 * 3892 = 15378880 (which is a product, not a sum, so"],
 ['<s><|user|> What is 3940 * 3892?<|end|><|assistant|> The product of 3940 and 3892 is 15270080.\n\n3940 * 3892 = 1527008',
  '<s><|user|> What is 3940 * 3892?<|end|><|assistant|> Here is one way to solve this problem using Python:\n\n```python\n# Define the numbers\nnum1 = 3940\nnum2 = 3892\n\n# Multiply the numbers',
  '<s><|user|> What is 3940 * 3892?<|end|><|assistant|> 14460800<|end|><|assistant|>Reply\nThe answer is: 14460800<|end|><|assistant|> Note: The original question seems to be a straightforward multiplication problem without any further context provided',
  '<s><|user|> What is 3940 * 3892?<|end|><|assistant|> The answer is 147,708,800. \n\nTo calculate this, you multiply the two numbers together:\n\n3940 * 3892 = 14'],
 ['<s><|user|> What is 3940 * 3892?<|end|><|assistant|> To calculate the product of 3940 and 3892, we perform long multiplication:\n\n```\n\n   3940\n\nx  3892\n\n------\n\n  ',
  '<s><|user|> What is 3940 * 3892?<|end|><|assistant|> To calculate the product of 3940 and 3892, you simply multiply the two numbers together.\n\n\n\\[\n\n3940 \\times 3892 = 1',
  '<s><|user|> What is 3940 * 3892?<|end|><|assistant|> To calculate 3940 multiplied by 3892, we can follow these steps:\n\n\n1. **Set up the multiplication:**\n\n   ```\n\n   3940\n',
  '<s><|user|> What is 3940 * 3892?<|end|><|assistant|> To find the product of 3940 and 3892, you would multiply these two numbers using long multiplication:\n\n\n```\n\n  3940\n\nx 3892'],
 ["<s><|user|> What is 3940 * 3892?

In [10]:
from micrlhf.utils.load_sae import get_sae
from micrlhf.utils.ito import grad_pursuit
resid_out = aux["resid_out"]
refusal = (resid_out[{"batch": -4}] - resid_out.untag("batch").mean()).unwrap("embedding")
sae = get_sae(20, 5)
k = 2
w, recon = grad_pursuit(refusal, sae["W_dec"], k, pos_only=True)
i = jax.lax.top_k(jnp.abs(w), k)[1]
((recon - refusal) ** 2).mean(), i, w[i]

--2024-05-30 04:37:42--  https://huggingface.co/nev/phi-3-4k-saex-test/resolve/main/l20-test-run-5-7.00E-06/sae_weights.safetensors?download=true
Resolving huggingface.co (huggingface.co)... 108.156.211.90, 108.156.211.95, 108.156.211.51, ...
Connecting to huggingface.co (huggingface.co)|108.156.211.90|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://cdn-lfs-us-1.huggingface.co/repos/eb/d8/ebd889d6ac58573e8e8a7aa1176d4d357581a6da60135b94aca378fddf4e9e54/83ac196e9d76d5a3db26b4cc47737ff5c79d3cba0ce3954d02283b9331464948?response-content-disposition=attachment%3B+filename*%3DUTF-8%27%27sae_weights.safetensors%3B+filename%3D%22sae_weights.safetensors%22%3B&Expires=1717303062&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTcxNzMwMzA2Mn19LCJSZXNvdXJjZSI6Imh0dHBzOi8vY2RuLWxmcy11cy0xLmh1Z2dpbmdmYWNlLmNvL3JlcG9zL2ViL2Q4L2ViZDg4OWQ2YWM1ODU3M2U4ZThhN2FhMTE3NmQ0ZDM1NzU4MWE2ZGE2MDEzNWI5NGFjYTM3OGZkZGY0ZTllNTQvODNhYzE5NmU5

(<jax.Array(23.02443, dtype=float32)>,
 <jax.Array([20279, 23629], dtype=int32)>,
 <jax.Array([44.070187, 39.419018], dtype=float32)>)

In [11]:
from micrlhf.utils.activation_manipulation import set_direction


layer = layer_target
repeat = 64
vector = refusal
# vector = recon
# act_add = add_vector(llama,
#                     (vector / jnp.linalg.norm(vector, axis=-1, keepdims=True))[None, :] * jnp.linspace(-200, 200, repeat)[:, None],
#                      layer)
act_add = set_direction(llama, vector, jnp.linspace(-100, 100, repeat), layer)

sample(act_add, tokenizer, prompt,
       batch_size=repeat, do_sample=False)

  0%|          | 0/46 [00:00<?, ?it/s]

['<s><|user|> What is 3940 * 3892?<|end|><|assistant|> The product of 3940 and 3892 is approximately 1, poorer-than-anything-I-think-of-in-response-to-your-question, but I',
 "<s><|user|> What is 3940 * 3892?<|end|><|assistant|> The product of 3940 and 3892 is approximately 1,199,551. However, without the exact calculation, I can't provide the precise total, but it'",
 '<s><|user|> What is 3940 * 3892?<|end|><|assistant|> The product of 3940 and 3892 is approximately 11,199,968. To get the exact answer, the product of 3940 and 38',
 "<s><|user|> What is 3940 * 3892?<|end|><|assistant|> The product of 3940 and 3892 is approximately 1,199,551. However, without the exact calculation, I can't provide the precise answer, but it'",
 '<s><|user|> What is 3940 * 3892?<|end|><|assistant|> The product of 3940 and 3892 is approximately 11,999,888. To get the exact answer, the product of 3940 and 38',
 '<s><|user|> What is 3940 * 3892?<|end|><|assistant|> The product of 3940 and 3892 is approximately 11,999,888. To calculate it exactly, you can multiply the two numbers:\n\n3940',
 '<s><|user|> What is 3940 * 3892?<|end|><|assistant|> The product of 3940 and 3892 is approximately 11,999,888. To get the exact answer, the calculation is:\n\n3940 * ',
 '<s><|user|> What is 3940 * 3892?<|end|><|assistant|> The product of 3940 and 3892 is approximately 11,993,328. To calculate it exactly, you can multiply the two numbers:\n\n3940',
 '<s><|user|> What is 3940 * 3892?<|end|><|assistant|> The product of 3940 and 3892 is approximately 11,993,328. To calculate it exactly, you can multiply the two numbers:\n\n3940',
 '<s><|user|> What is 3940 * 3892?<|end|><|assistant|> The product of 3940 and 3892 is approximately 11,059,368. To calculate it exactly, you can multiply the two numbers:\n\n3940',
 '<s><|user|> What is 3940 * 3892?<|end|><|assistant|> The product of 3940 and 3892 is 11 breeding ground for mosquitoes. However, the actual product of those numbers is not related to mosquitoes. The product',
 "<s><|user|> What is 3940 * 3892?<|end|><|assistant|> The product of 3940 and 3892 is 11,139,360. Without knowing the specific numbers you're asking about, I can't provide a more detailed",
 "<s><|user|> What is 3940 * 3892?<|end|><|assistant|> The product of 3940 and 3892 is 1,173,068. Without a calculator, it's hard to calculate that quickly, but if you're interested",
 '<s><|user|> What is 3940 * 3892?<|end|><|assistant|> The product of 3940 and 3892 is 1,1 breeding 9,408, which is not possible as the product of two numbers is always a specific value. However,',
 '<s><|user|> What is 3940 * 3892?<|end|><|assistant|> The product of 3940 and 3892 is 1,1 breeding 9,408,720.\n\nTo calculate it, you can multiply the two numbers:\n',
 '<s><|user|> What is 3940 * 3892?<|end|><|assistant|> The product of 3940 and 3892 is 1,1 breeding 9,408,720.\n\nTo calculate it, you can multiply the two numbers:\n',
 '<s><|user|> What is 3940 * 3892?<|end|><|assistant|> The product of 3940 and 3892 is 1,149,328. To calculate this, you can multiply the two numbers:\n\n3940 * 3',
 '<s><|user|> What is 3940 * 3892?<|end|><|assistant|> The product of 3940 and 3892 is 1,146,236. To calculate this, you can multiply the two numbers:\n\n3940 * 3',
 '<s><|user|> What is 3940 * 3892?<|end|><|assistant|> The product of 3940 and 3892 is 1,143,328. To calculate this, you can multiply the two numbers:\n\n3940 * 3',
 "<s><|user|> What is 3940 * 3892?<|end|><|assistant|> The product of 3940 and 3892 is 15364320.<|end|><|assistant|> Yes, that's correct! The product of 3940 and 38",
 "<s><|user|> What is 3940 * 3892?<|end|><|assistant|> The product of 3940 and 3892 is 15364320.<|end|><|assistant|> Yes, that's correct. The calculation is as follows:\n\n3940",
 "<s><|user|> What is 3940 * 3892?<|end|><|assistant|> The product of 3940 and 3892 is 15363360.<|end|><|assistant|> Yes, that's correct. The calculation is as follows:\n\n3940",
 "<s><|user|> What is 3940 * 3892?<|end|><|assistant|> T

In [12]:
from micrlhf.utils.load_sae import get_sae
from micrlhf.utils.ito import grad_pursuit
refusal_steer = addition[-4]
sae_steer = get_sae(10, 4)
k = 16
w, recon_steer = grad_pursuit(refusal_steer, sae_steer["W_dec"], k, pos_only=True)
i = jax.lax.top_k(jnp.abs(w), k)[1]
((recon_steer - refusal_steer) ** 2).mean(), i, w[i]

--2024-05-30 04:39:06--  https://huggingface.co/nev/phi-3-4k-saex-test/resolve/main/l10-test-run-4-2.73E-06/sae_weights.safetensors?download=true
Resolving huggingface.co (huggingface.co)... 108.156.211.95, 108.156.211.51, 108.156.211.90, ...
Connecting to huggingface.co (huggingface.co)|108.156.211.95|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://cdn-lfs-us-1.huggingface.co/repos/eb/d8/ebd889d6ac58573e8e8a7aa1176d4d357581a6da60135b94aca378fddf4e9e54/c8cf3f62062b3cdc9509167526de244fa3271fca14dd9dc97f653ef970374bd7?response-content-disposition=attachment%3B+filename*%3DUTF-8%27%27sae_weights.safetensors%3B+filename%3D%22sae_weights.safetensors%22%3B&Expires=1717303146&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTcxNzMwMzE0Nn19LCJSZXNvdXJjZSI6Imh0dHBzOi8vY2RuLWxmcy11cy0xLmh1Z2dpbmdmYWNlLmNvL3JlcG9zL2ViL2Q4L2ViZDg4OWQ2YWM1ODU3M2U4ZThhN2FhMTE3NmQ0ZDM1NzU4MWE2ZGE2MDEzNWI5NGFjYTM3OGZkZGY0ZTllNTQvYzhjZjNmNjIw

(<jax.Array(1.0404294, dtype=float32)>,
 # jax.Array int32(16,) [≥694, ≤46_558] nonzero:16
   Array([38392, 21968,  7395, 35911, 46558, 17895,  8265, 30491, 37354,
          19255, 13932, 11776,  9762,   694, 41052, 14909], dtype=int32)
 ,
 # jax.Array float32(16,) ≈3.8 ±0.41 [≥3.4, ≤5.0] nonzero:16
   Array([4.993171 , 4.415919 , 4.1850085, 3.9498377, 3.9282415, 3.8214664,
          3.8211484, 3.7118008, 3.6665282, 3.6618662, 3.5518913, 3.4981477,
          3.4750347, 3.469136 , 3.4441085, 3.4338064], dtype=float32)
 )

In [13]:
layer = layer_source
repeat = 64
# vector = addition[-4]
vector = recon_steer
act_add = add_vector(llama,
                     (vector / jnp.linalg.norm(vector, axis=-1, keepdims=True))[None, :] * jnp.linspace(-scale * 2, scale * 2, repeat)[:, None],
                     layer)

sample(act_add, tokenizer, prompt,
       batch_size=repeat, do_sample=False)

  0%|          | 0/46 [00:00<?, ?it/s]

['<s><|user|> What is 3940 * 3892?<|end|><|assistant|> To calculate the product of 3940 and 3892, you can use the standard multiplication method:\n\n3940\nx 3892\n-----\nFirst, multiply the ones',
 '<s><|user|> What is 3940 * 3892?<|end|><|assistant|> To calculate the product of 3940 and 3892, you can use the standard multiplication method:\n\n3940\nx 3892\n-----\nFirst, multiply the ones',
 '<s><|user|> What is 3940 * 3892?<|end|><|assistant|> To calculate the product of 3940 and 3892, you can use the standard multiplication method:\n\n3940\nx 3892\n-----\nFirst, multiply 3',
 '<s><|user|> What is 3940 * 3892?<|end|><|assistant|> To calculate the product of 3940 and 3892, you can use the standard multiplication method:\n\n3940\nx 3892\n-----\nFirst, multiply 3',
 '<s><|user|> What is 3940 * 3892?<|end|><|assistant|> To calculate the product of 3940 and 3892, you can use the standard multiplication method:\n\n3940\nx 3892\n-----\n15760',
 '<s><|user|> What is 3940 * 3892?<|end|><|assistant|> To calculate the product of 3940 and 3892, you can use the standard multiplication method:\n\n3940\nx 3892\n-----\n15760',
 '<s><|user|> What is 3940 * 3892?<|end|><|assistant|> To calculate the product of 3940 and 3892, you can use the standard multiplication method:\n\n3940\nx 3892\n-----\n15760',
 '<s><|user|> What is 3940 * 3892?<|end|><|assistant|> To calculate the product of 3940 and 3892, you can use the standard multiplication method:\n\n3940\nx 3892\n-----\n15760',
 '<s><|user|> What is 3940 * 3892?<|end|><|assistant|> To calculate the product of 3940 and 3892, you can use the standard multiplication method:\n\n3940\nx 3892\n-----\n15760',
 '<s><|user|> What is 3940 * 3892?<|end|><|assistant|> To calculate the product of 3940 and 3892, you can use the standard multiplication method:\n\n3940\nx 3892\n-----\n15760',
 '<s><|user|> What is 3940 * 3892?<|end|><|assistant|> To calculate the product of 3940 and 3892, you can use the standard multiplication method:\n\n3940\nx 3892\n-----\n15720',
 '<s><|user|> What is 3940 * 3892?<|end|><|assistant|> To calculate the product of 3940 and 3892, you can use the standard multiplication method:\n\n3940\nx 3892\n_____\n1576',
 '<s><|user|> What is 3940 * 3892?<|end|><|assistant|> To calculate the product of 3940 and 3892, you can use the standard multiplication method:\n\n3940\nx 3892\n_____\n1576',
 '<s><|user|> What is 3940 * 3892?<|end|><|assistant|> To calculate the product of 3940 and 3892, you can use the standard multiplication method:\n\n3940\nx 3892\n_____\n1576',
 '<s><|user|> What is 3940 * 3892?<|end|><|assistant|> To calculate the product of 3940 and 3892, you can use the standard multiplication method:\n\n3940\nx 3892\n_____\n1572',
 '<s><|user|> What is 3940 * 3892?<|end|><|assistant|> To calculate the product of 3940 and 3892, you can use the standard multiplication method:\n\n3940\nx 3892\n_____\n1576',
 '<s><|user|> What is 3940 * 3892?<|end|><|assistant|> To calculate the product of 3940 and 3892, you can use the standard multiplication method:\n\n3940\nx 3892\n_____\n1572',
 '<s><|user|> What is 3940 * 3892?<|end|><|assistant|> To calculate the product of 3940 and 3892, you can use the standard multiplication method:\n\n3940\nx 3892\n_____\n1572',
 '<s><|user|> What is 3940 * 3892?<|end|><|assistant|> The product of 3940 and 3892 is 15,361,280. You can calculate this by multiplying the two numbers together:\n\n3940\n',
 '<s><|user|> What is 3940 * 3892?<|end|><|assistant|> The product of 3940 and 3892 is 15,361,280. You can calculate this by multiplying the two numbers together:\n\n3940\n',
 '<s><|user|> What is 3940 * 3892?<|end|><|assistant|> The product of 3940 and 3892 is 15,365,280. You can calculate this by multiplying the two numbers together:\n\n3940\n',
 '<s><|user|> What is 3940 * 3892?<|end|><|assistant|> The product of 3940 and 3892 is 15,361,280. You can calculate this by multiplying the two numbers together:\n\n3940\n',
 '<s><|user|> What is 3940 * 3892?<|end|>

: 